In [1]:
api_key = ""
channel_id = ""

In [2]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [3]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

In [4]:
videos = get_channel_videos(channel_id)

In [5]:
res = youtube.videos().list(id=videos[0]["snippet"]["resourceId"]["videoId"],
                           part="statistics").execute()

In [6]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [7]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [8]:
stats = get_videos_stats(video_ids)

In [20]:
stats[100]

{'kind': 'youtube#video',
 'etag': '7Rc1TWwxcWVok6RWwJUqVKlbo64',
 'id': 'gRQ2Db06QMg',
 'statistics': {'viewCount': '11012',
  'likeCount': '442',
  'dislikeCount': '10',
  'favoriteCount': '0',
  'commentCount': '35'}}

# likeCount와 dislikeCount와 commentCount

In [42]:
publishedAt = []
title = []
viewCount = []
likeCount = []
dislikeCount = []
commentCount = []

for i in range(len(videos)):
    publishedAt.append(videos[i]['snippet']['publishedAt'])
    title.append(videos[i]['snippet']['title'])
    viewCount.append(stats[i]['statistics']['viewCount'])
    likeCount.append(stats[i]['statistics']['likeCount'])
    dislikeCount.append(stats[i]['statistics']['dislikeCount'])
    commentCount.append(stats[i]['statistics']['commentCount'])

In [43]:
dict = {}
dict["publishedAt"] = publishedAt
dict["title"] = title
dict["viewCount"] = viewCount
dict['likeCount'] = likeCount
dict['dislikeCount'] = dislikeCount
dict['commentCount'] = commentCount

In [44]:
import pandas as pd
df = pd.DataFrame.from_dict(dict)

# int로 변환

In [45]:
df['viewCount'] = df['viewCount'].astype(int)
df['likeCount'] = df['likeCount'].astype(int)
df['dislikeCount'] = df['dislikeCount'].astype(int)
df['commentCount'] = df['commentCount'].astype(int)

In [46]:
df.head()

,publishedAt,title,viewCount,likeCount,dislikeCount,commentCount
0,2021-04-09T21:00:13Z,에이치엘비 진양곤회장이 기업을 인수하면 모든 게 확 바뀐다!!,1854,199,8,12
1,2021-04-09T09:00:28Z,에이치엘비 뇌종양 항암효과 네이처 자매지에 실렸다!,9280,472,12,81
2,2021-04-09T03:00:10Z,에이치엘비 증선위가 무증 효과를 지웠다!!,7701,329,7,39
3,2021-04-08T21:00:31Z,에이치엘비 베리스모 CAR-T 중국 시장 진출 본격화하면 현재 긴밀한 관계를 맺고 ...,5181,311,4,38
4,2021-04-08T08:00:09Z,에이치엘비 CAR-T 상업화 최소 3년이상 앞당길 수 있다!! CAR-T세포가 활성...,6944,336,7,35


In [14]:
len(df)

2595

# 좋아요/싫어요 비율

In [69]:
# 0으로 나누는 걸 방지하기 위해 0.001을 더했다.
df['likeRatio'] = df['likeCount'] / (df['likeCount'] + df['dislikeCount'] + 0.01)

In [70]:
df.head()

,publishedAt,title,viewCount,likeCount,dislikeCount,commentCount,likeRatio
0,2021-04-09T21:00:13Z,에이치엘비 진양곤회장이 기업을 인수하면 모든 게 확 바뀐다!!,1854,199,8,12,0.961306
1,2021-04-09T09:00:28Z,에이치엘비 뇌종양 항암효과 네이처 자매지에 실렸다!,9280,472,12,81,0.975186
2,2021-04-09T03:00:10Z,에이치엘비 증선위가 무증 효과를 지웠다!!,7701,329,7,39,0.979138
3,2021-04-08T21:00:31Z,에이치엘비 베리스모 CAR-T 중국 시장 진출 본격화하면 현재 긴밀한 관계를 맺고 ...,5181,311,4,38,0.987270
4,2021-04-08T08:00:09Z,에이치엘비 CAR-T 상업화 최소 3년이상 앞당길 수 있다!! CAR-T세포가 활성...,6944,336,7,35,0.979563


# 제목에 '에이치엘비'가 없다면 Drop하기

In [71]:
df['에이치엘비'] = df['title'].str.contains('에이치엘비')
df = df.loc[(df['에이치엘비'] == True)]
len(df)

1449

# 제목에 '에이치엘비생명과학'이 있다면 Drop하기

In [72]:
df['에이치엘비생명과학'] = df['title'].str.contains('에이치엘비생명과학')
df = df.loc[df['에이치엘비생명과학'] == False]
len(df)

1412

# 제목에 '에이치엘비제약'이 있다면 Drop하기

In [73]:
df['에이치엘비제약'] = df['title'].str.contains('에이치엘비제약')
df = df.loc[(df['에이치엘비제약'] == False)]
len(df)

1384

# 과거 절반 데이터 Drop하기

In [74]:
df_half = df.copy()
df_half = df_half[:int(len(df_half)/2)]
len(df_half)

692

# 최근 데이터 Drop하기
공개된지 몇 시간 되지 않아 아직 조회수가 채워지지 않았기 때문에 같은 기준으로 분석할 수 없다.

In [75]:
df_half = df_half[5:]

In [76]:
len(df_half)

687

In [78]:
df_half.head()

,publishedAt,title,viewCount,likeCount,dislikeCount,commentCount,likeRatio,에이치엘비,에이치엘비생명과학,에이치엘비제약
5,2021-04-08T01:34:11Z,에이치엘비 주주 오세훈 서울시장 당선!!,6070,339,34,48,0.908823,True,False,False
6,2021-04-07T21:00:15Z,에이치엘비 진양곤 회장은 코르키로부터 확보했던 넥스트사이언스의 메자닌에 대한 콜옵션...,8479,354,7,35,0.980582,True,False,False
7,2021-04-07T08:15:01Z,에이치엘비 증선위 뉴스기사가 없는 것으로 보아 열리지 않고 무기한 연기 가능성!!,7676,321,7,38,0.978629,True,False,False
8,2021-04-07T03:00:11Z,에이치엘비가 뛰어든 CAR-T치료제의 부작용은 노바티스와 길리어드사이언스에서도 보이...,3196,181,3,16,0.983642,True,False,False
9,2021-04-06T21:00:09Z,에이치엘비 CAR-T는 암환자 숫자의 90% 이상인 고형암을 공략하고 노바티스 CA...,5508,325,3,31,0.990823,True,False,False


# 분석을 위해 숫자 데이터만 남기고 Drop하기

In [82]:
df_analysis = df_half[['viewCount','likeCount','dislikeCount','commentCount','likeRatio']]

In [83]:
df_analysis.head()

,viewCount,likeCount,dislikeCount,commentCount,likeRatio
5,6070,339,34,48,0.908823
6,8479,354,7,35,0.980582
7,7676,321,7,38,0.978629
8,3196,181,3,16,0.983642
9,5508,325,3,31,0.990823


# 상관관계 구하기
참고 링크 : https://gomguard.tistory.com/173

In [85]:
df_analysis.corr(method='pearson')

,viewCount,likeCount,dislikeCount,commentCount,likeRatio
viewCount,1.000000,0.854699,0.638831,0.773863,-0.071445
likeCount,0.854699,1.000000,0.585636,0.786839,0.079149
dislikeCount,0.638831,0.585636,1.000000,0.539844,-0.660223
commentCount,0.773863,0.786839,0.539844,1.000000,-0.007406
likeRatio,-0.071445,0.079149,-0.660223,-0.007406,1.000000


# Results
viewCount와 상관관계가 높은 지표는 순서대로 likeCount(0.85), commentCount(0.77)이 있다.  
즉, 좋아요와 댓글이 많으면 조회수가 높다. 조회수가 높으면 좋아요와 댓글이 많다.

# Conclusion
좋아요수와 댓글수는 조회수와 강한 상관관계를 갖는다. 좋아요수와 댓글수를 높이는 노력을 한다.  
하지만 이는 상관관계이므로 향후 인과관계를 밝혀내기 위한 연구와 실험을 해야 할 것이다.  
  
경험적으로 조회수가 높은 영상은 전체 공개 후 빠르면 10분, 늦으면 1시간 이내 조회수가 얼마까지 갈지 예측할 수 있었다.  
그 이유는 아마도 조회수 증가속도, 좋아요수, 댓글수가 선행지표이기 때문이다.  
예를 들어, 조회수 증가속도가 2배인가 3배인가에 따라 예측되는 최종 조회수가 있다.  
마찬가지로, 좋아요수가 몇 개인가에 따라 역시 다른 영상 조회수와 비교하여 예측되는 최종 조회수가 있다.  
중요한 사실은 아직 조회수가 점근선에 도달하지 않았을 때 미리 알 수 있다는 사실이다.  
경험적인 근거가 있으나 숫자 데이터로 확인하기까지는 확신할 수 없다. 계속 연구를 진행한다.